In [1]:
# ===================================================================
# 1. IMPORTAR LIBRERÍAS
# ===================================================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Configurar seaborn para que las gráficas se vean mejor
sns.set()

# ===================================================================
# 2. CARGAR Y EXPLORAR EL DATASET
# ===================================================================

# Cargar el dataset
iris = load_iris()

# Crear un DataFrame de Pandas para mejor manejo
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['species_id'] = iris.target
df['species'] = df['species_id'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

# --- Requisito: "utilice las primeras 5 muestras del dataset" ---
print("--- 1. Primeras 5 muestras del dataset 'iris' ---")
print(df.head(5))
print("\n" + "="*50 + "\n")

# ===================================================================
# 3. DIVIDIR DATOS (ENTRENAMIENTO Y PRUEBA)
# ===================================================================

# Definir X (features) e y (target)
X = df.drop(['species_id', 'species'], axis=1)
y = df['species_id'] # Usamos el ID (0, 1, 2)

# Dividir en set de entrenamiento (70%) y prueba (30%)
# random_state=42 asegura que siempre obtengamos la misma división
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# ===================================================================
# 4. ENCONTRAR EL MEJOR VALOR DE K
# ===================================================================

print("--- 2. Encontrando el mejor valor de K ---")
# Rango de K que queremos probar
k_range = range(1, 21)
accuracies = [] # Lista para guardar la precisión de cada K

for k in k_range:
    # Crear y entrenar el modelo K-NN
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    # Hacer predicciones en el set de prueba
    y_pred = knn.predict(X_test)
    
    # Calcular y guardar la precisión
    accuracies.append(accuracy_score(y_test, y_pred))

# Encontrar el mejor K (el que dio la mayor precisión)
best_k = k_range[accuracies.index(max(accuracies))]
print(f"La mayor precisión se encontró con K = {best_k} (Precisión: {max(accuracies):.4f})")

# --- Gráfica del Método del Codo/Precisión ---
plt.figure(figsize=(10, 6))
plt.plot(k_range, accuracies, marker='o', linestyle='dashed')
plt.title('Precisión del Modelo K-NN vs. Valor de K')
plt.xlabel('Valor de K')
plt.ylabel('Precisión (Accuracy)')
plt.xticks(k_range)
plt.grid(True)
plt.show()

print("\n" + "="*50 + "\n")

# ===================================================================
# 5. MODELO FINAL Y PREDICCIÓN DE LAS 5 PRIMERAS MUESTRAS
# ===================================================================

# Entrenar el modelo final con el mejor K encontrado
modelo_final_knn = KNeighborsClassifier(n_neighbors=best_k)
modelo_final_knn.fit(X_train, y_train) # Entrenamos con los datos de entrenamiento

# --- Requisito: Predicción de las primeras 5 muestras ---
# Seleccionamos las características de las primeras 5 filas
primeras_5_features = X.iloc[:5] 
# Seleccionamos las etiquetas reales de esas 5 filas
primeras_5_reales = y.iloc[:5].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

# Usamos el modelo entrenado para predecir
prediccion_5 = modelo_final_knn.predict(primeras_5_features)
prediccion_5_nombres = [iris.target_names[p] for p in prediccion_5]

print("--- 3. Predicción de las primeras 5 muestras ---")
print(f"Valores Reales: {list(primeras_5_reales)}")
print(f"Predicciones del Modelo: {prediccion_5_nombres}")
print("\n" + "="*50 + "\n")


# ===================================================================
# 6. GRÁFICA DE LA CLASIFICACIÓN CREADA POR KNN
# ===================================================================
print("--- 4. Generando gráfica de clasificación K-NN ---")

# Obtener las predicciones para TODO el set de prueba
y_pred_test = modelo_final_knn.predict(X_test)

# Convertir a DataFrame para facilitar la gráfica con Seaborn
df_test = X_test.copy()
df_test['species_real'] = y_test.map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})
df_test['species_pred'] = pd.Series(y_pred_test, index=df_test.index).map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

# --- Gráfica de Comparación: Real vs. Predicha ---
fig, axes = plt.subplots(1, 2, figsize=(16, 7), sharey=True)

# Gráfica 1: Clasificación Real
sns.scatterplot(
    ax=axes[0],
    data=df_test,
    x='petal length (cm)',
    y='petal width (cm)',
    hue='species_real',
    style='species_real',
    palette='deep',
    s=100
)
axes[0].set_title('Clasificación Real (Datos de Prueba)')

# Gráfica 2: Clasificación Predicha por K-NN
sns.scatterplot(
    ax=axes[1],
    data=df_test,
    x='petal length (cm)',
    y='petal width (cm)',
    hue='species_pred',
    style='species_pred',
    palette='deep',
    s=100
)

axes[1].set_title(f'Clasificación Predicha por K-NN (K={best_k})')

plt.suptitle('Comparación de Clasificación Real vs. Predicha', fontsize=16)
plt.show()

print("--- Ejecución Completa ---")

ModuleNotFoundError: No module named 'pandas'